# Introduction to Snowpark pandas
The Snowpark pandas API allows you to run your pandas code directly on your data in Snowflake. Built to replicate the functionality of pandas - including its data isolation and consistency guarantees - the Snowpark pandas API enables you to scale up your traditional pandas pipelines with just a few lines of change.

In today's demo, we'll be taking a look at how you can get started with the API, as well as comparing its simpilarity with vanilla pandas.

## Importing Snowpark pandas
Much like Snowpark, Snowpark pandas requires an active `Session` object to connect to your data in Snowflake. In the next cell, we'll be initializing a Session object, and importing both Snowpark pandas and vanilla pandas, as `spd` and `pd` respectively.

In [1]:
import snowflake.snowpark.modin.pandas as spd
import pandas as pd
import json
from snowflake.snowpark.session import Session
# Create Snowflake Session object
# connection_parameters = json.load(open('creds.json'))
session = Session.builder.create()

In [2]:
spd.DataFrame([1])

,0
0,1


## Getting Started - Reading Data from Snowflake
Today, we'll be analyzing some Stock Timeseries Data from Snowflake's Marketplace. The data is available courtesy of Cybersyn Inc., and can be found [here](https://app.snowflake.com/marketplace/listing/GZTSZAS2KF7/cybersyn-inc-financial-economic-essentials). Let's start by reading the `stock_price_timeseries` table into a DataFrame!

In [3]:
# Read data into a Snowpark pandas df 
from time import perf_counter
start = perf_counter()
spd_df = spd.read_snowflake("FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.STOCK_PRICE_TIMESERIES")
end = perf_counter()
data_size = len(spd_df)
print(f"Took {end - start} seconds to read a table with {data_size} rows into Snowpark pandas!")
snow_time = end - start

Took 12.530617542 seconds to read a table with 65921095 rows into Snowpark pandas!


In [4]:
# Read data into a local Vanilla pandas df - recommended to kill this cell after waiting a few minutes!
# Create a cursor object.
from IPython import display
start = perf_counter()

# Create a cursor object.
cur = session.connection.cursor()

# Execute a statement that will generate a result set.
sql = "select * from FINANCIAL__ECONOMIC_ESSENTIALS.CYBERSYN.STOCK_PRICE_TIMESERIES"
cur.execute(sql)

# Fetch the result set from the cursor and deliver it as the pandas DataFrame.
native_pd_df = cur.fetch_pandas_all()
end = perf_counter()
print(f"Vanilla pandas took {end - start} seconds to read the data!")

Vanilla pandas took 353.29520379099995 seconds to read the data!


As we can see, pandas is not able to pull the data into memory even after being given a few minutes.

## Examine The Raw Data
Let's take a look at the data we're going to be working with

In [5]:
spd_df.head(5).to_pandas()

,TICKER,ASSET_CLASS,PRIMARY_EXCHANGE_CODE,PRIMARY_EXCHANGE_NAME,VARIABLE,VARIABLE_NAME,DATE,VALUE
0,SCWX,Equity,NAS,NASDAQ CAPITAL MARKET,nasdaq_volume,Nasdaq Volume,2019-02-22,47099.0
1,PUK,Equity,NYS,NEW YORK STOCK EXCHANGE,nasdaq_volume,Nasdaq Volume,2019-05-21,35378.0
2,PAA,Equity,NAS,NASDAQ CAPITAL MARKET,nasdaq_volume,Nasdaq Volume,2021-07-15,622778.0
3,MSFT,Equity,NAS,NASDAQ CAPITAL MARKET,nasdaq_volume,Nasdaq Volume,2018-09-18,7682704.0
4,GRIF,Equity,NAS,NASDAQ CAPITAL MARKET,nasdaq_volume,Nasdaq Volume,2019-10-24,341.0


In [6]:
native_pd_df.head(5)

,TICKER,ASSET_CLASS,PRIMARY_EXCHANGE_CODE,PRIMARY_EXCHANGE_NAME,VARIABLE,VARIABLE_NAME,DATE,VALUE
0,RFP,Equity,NYS,NEW YORK STOCK EXCHANGE,all-day_high,All-Day High,2020-05-12,2.095
1,CMCTP,Equity,NAS,NASDAQ CAPITAL MARKET,all-day_high,All-Day High,2020-06-05,23.510
2,TUP,Equity,NYS,NEW YORK STOCK EXCHANGE,all-day_high,All-Day High,2020-06-05,6.940
3,HWM,Equity,NYS,NEW YORK STOCK EXCHANGE,all-day_high,All-Day High,2021-01-04,28.730
4,ALIM,Equity,NAS,NASDAQ CAPITAL MARKET,all-day_high,All-Day High,2018-10-19,1.000


## Filtering The Data
Let's take a look at some common data transformations - starting with filtering! Let's filter for stocks that are listed on the New York Stock Exchange!

In [7]:
start = perf_counter()
nyse_spd_df = spd_df[(spd_df['PRIMARY_EXCHANGE_CODE'] == 'NYS')]
repr(nyse_spd_df)
end = perf_counter()
print(f"Filtering for stocks belonging to the NYSE took {end - start} seconds in Snowpark pandas")

Filtering for stocks belonging to the NYSE took 3.6832122920000074 seconds in Snowpark pandas


In [8]:
start = perf_counter()
nyse_native_df = native_pd_df[(native_pd_df['PRIMARY_EXCHANGE_CODE'] == 'NYS')]
repr(nyse_native_df)
end = perf_counter()
print(f"Filtering for stocks belonging to the NYSE took {end - start} seconds in native pandas")

Filtering for stocks belonging to the NYSE took 3.289150999999947 seconds in native pandas


Let's try an even more granular filter - let's filter for the Pre-Market Open of stocks that have the following tickers:
* GOOG (Alphabet, Inc.)
* MSFT (Microsoft)
* SNOW (Snowflake)

In [9]:
start = perf_counter()
filtered_spd_df = spd_df[((spd_df['TICKER'] == 'GOOG') | (spd_df['TICKER'] == 'MSFT') | (spd_df['TICKER'] == 'SNOW')) & (spd_df['VARIABLE_NAME'] == 'Pre-Market Open')]
repr(filtered_spd_df)
end = perf_counter()
print(f"Filtering for the Pre-Market Open price for the above stocks belonging took {end - start} seconds in Snowpark pandas")

Filtering for the Pre-Market Open price for the above stocks belonging took 1.1148754999999824 seconds in Snowpark pandas


In [10]:
start = perf_counter()
filtered_native_df = native_pd_df[((native_pd_df['TICKER'] == 'GOOG') | (native_pd_df['TICKER'] == 'MSFT') | (native_pd_df['TICKER'] == 'SNOW')) & (native_pd_df['VARIABLE_NAME'] == 'Pre-Market Open')]
repr(filtered_native_df)
end = perf_counter()
print(f"Filtering for the Pre-Market Open price for the above stocks belonging took {end - start} seconds in Snowpark pandas")

Filtering for the Pre-Market Open price for the above stocks belonging took 7.6032268750000185 seconds in Snowpark pandas


# Reshaping the Data
Let's say we wanted to analyse the performance of various stock prices across time - in that case, it may be more helpful to have the values as columns, and the ticker name and date as the index - rather than the current encoding. We can accomplish this using the `pivot_table` API!

In [11]:
start = perf_counter()
reshape_df = spd_df.pivot_table(index=["TICKER", "DATE"], columns="VARIABLE_NAME", values="VALUE")
repr(reshape_df)
end = perf_counter()
print(f"Pivoting the DataFrame took {end - start} seconds in Snowpark pandas")

Pivoting the DataFrame took 15.539888792 seconds in Snowpark pandas


In [12]:
reshape_df.head(5).to_pandas()

VARIABLE_NAME      All-Day High  All-Day Low  Nasdaq Volume  \
TICKER DATE                                                   
A      2018-05-01         66.35        65.50       439231.0   
       2018-05-02         66.86        65.81       316586.0   
       2018-05-03         66.46        64.85       407491.0   
       2018-05-04         67.25        65.63       269025.0   
       2018-05-07         67.98        67.08       263454.0   

VARIABLE_NAME      Post-Market Close  Pre-Market Open  
TICKER DATE                                            
A      2018-05-01              66.23            65.64  
       2018-05-02              65.91            66.01  
       2018-05-03              66.33            65.91  
       2018-05-04              66.99            66.06  
       2018-05-07              67.40            67.16

In [13]:
start = perf_counter()
reshape_native_df = native_pd_df.pivot_table(index=["TICKER", "DATE"], columns="VARIABLE_NAME", values="VALUE")
repr(reshape_native_df)
end = perf_counter()
print(f"Pivoting the DataFrame took {end - start} seconds in native pandas")

Pivoting the DataFrame took 27.210994709000033 seconds in native pandas


## Transforming the Data
Now that we have reformatted the data, we can beginn to apply some transformations. Let's start by taking a look at the All-Day Low column for the tickers above - we can resample the data to look at the Quarterly Low for the `GOOG` ticker!

In [14]:
start = perf_counter()
resampled_spd_df_all_quarter_low = reshape_df["All-Day Low"]["GOOG"].resample("91D").min()
repr(resampled_spd_df_all_quarter_low)
end = perf_counter()
print(f"Resampling the DataFrame took {end - start} seconds in Snowpark pandas")

Resampling the DataFrame took 2.4453825420000044 seconds in Snowpark pandas


In [15]:
resampled_spd_df_all_quarter_low

DATE
2018-05-01    1006.48
2018-07-31     995.58
2018-10-30     968.09
2019-01-29    1055.85
2019-04-30    1025.06
2019-07-30    1125.00
2019-10-29    1250.79
2020-01-28    1013.54
2020-04-28    1218.04
2020-07-28    1399.96
2020-10-27    1514.61
2021-01-26    1801.22
2021-04-27    2223.89
2021-07-27    2623.00
2021-10-26    2493.01
2022-01-25    2363.60
2022-04-26     107.01
2022-07-26      94.93
2022-10-25      83.00
2023-01-24      88.87
2023-04-25     101.66
2023-07-25     121.54
2023-10-24     121.47
2024-01-23     131.54
2024-04-23     152.77
Freq: None, Name: All-Day Low, dtype: float64

In [16]:
pandas_goog_frame = reshape_native_df["All-Day Low"]["GOOG"]
# native pandas checks that index is of date type. snowpark pandas does not.
pandas_goog_frame.index = pandas_goog_frame.index.astype('datetime64[ns]')
start = perf_counter()
resampled_native_df_all_quarter_low = pandas_goog_frame.resample("91D").min()
repr(resampled_native_df_all_quarter_low)
end = perf_counter()
print(f"Resampling the DataFrame took {end - start} seconds in native pandas")

Resampling the DataFrame took 0.001528500000006261 seconds in native pandas


We can even take a look at the quarter-over-quarter fluctuation in prices using the `diff` API!

In [17]:
start = perf_counter()
q_o_q_resampled_spd_df_all_quarter_low = resampled_spd_df_all_quarter_low.diff()
repr(q_o_q_resampled_spd_df_all_quarter_low)
end = perf_counter()
print(f"diffing the DataFrame took {end - start} seconds in Snowpark pandas")

diffing the DataFrame took 0.7945731249999994 seconds in Snowpark pandas


In [18]:
q_o_q_resampled_spd_df_all_quarter_low

DATE
2018-05-01        NaN
2018-07-31     -10.90
2018-10-30     -27.49
2019-01-29      87.76
2019-04-30     -30.79
2019-07-30      99.94
2019-10-29     125.79
2020-01-28    -237.25
2020-04-28     204.50
2020-07-28     181.92
2020-10-27     114.65
2021-01-26     286.61
2021-04-27     422.67
2021-07-27     399.11
2021-10-26    -129.99
2022-01-25    -129.41
2022-04-26   -2256.59
2022-07-26     -12.08
2022-10-25     -11.93
2023-01-24       5.87
2023-04-25      12.79
2023-07-25      19.88
2023-10-24      -0.07
2024-01-23      10.07
2024-04-23      21.23
Freq: None, Name: All-Day Low, dtype: float64

In [19]:
start = perf_counter()
q_o_q_resampled_native_df_all_quarter_low = resampled_native_df_all_quarter_low.diff()
repr(q_o_q_resampled_native_df_all_quarter_low)
end = perf_counter()
print(f"diffing the DataFrame took {end - start} seconds in native pandas")

diffing the DataFrame took 0.0007652909999933399 seconds in native pandas


In [20]:
q_o_q_resampled_native_df_all_quarter_low

DATE
2018-05-01        NaN
2018-07-31     -10.90
2018-10-30     -27.49
2019-01-29      87.76
2019-04-30     -30.79
2019-07-30      99.94
2019-10-29     125.79
2020-01-28    -237.25
2020-04-28     204.50
2020-07-28     181.92
2020-10-27     114.65
2021-01-26     286.61
2021-04-27     422.67
2021-07-27     399.11
2021-10-26    -129.99
2022-01-25    -129.41
2022-04-26   -2256.59
2022-07-26     -12.08
2022-10-25     -11.93
2023-01-24       5.87
2023-04-25      12.79
2023-07-25      19.88
2023-10-24      -0.07
2024-01-23      10.07
2024-04-23      21.23
Freq: 91D, Name: All-Day Low, dtype: float64

In [21]:
display.Markdown(data=f"""## Conclusion\nAs we can see, Snowpark pandas is able to replicate the pandas API while performing computations on large data sets that don't typically work with vanilla pandas and all while keeping your data in Snowflake!""")

## Conclusion
As we can see, Snowpark pandas is able to replicate the pandas API while performing computations on large data sets that don't typically work with vanilla pandas and all while keeping your data in Snowflake!